In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer

In [ ]:
from sklearn.linear_model import LinearRegression,Ridge,SGDRegressor,ElasticNet
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.utils.validation import check_array 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.svm import LinearSVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,ExtraTreesRegressor,GradientBoostingRegressor,AdaBoostRegressor,BaggingRegressor

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Loading Dataset**

In [ ]:
h = pd.read_csv(r'/content/drive/MyDrive/WiDS/DataSets/matches.csv')
ballData = pd.read_csv(r'/content/drive/MyDrive/WiDS/DataSets/deliveries.csv')

In [ ]:
h.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
ballData.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,NaN,NaN,NaN
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,NaN,NaN,NaN


In [ ]:
ballData.drop(ballData[ballData.inning.isin([3,4,5])].index, inplace=True)

Adding Features "Cummulaive Runs", "is_wicket", "cummulative wickets" and "score" to the dataframe

In [ ]:
l = []

for match in list(ballData['match_id'].unique()) :
    temp1 = ballData[ballData['match_id']==match][ballData['inning']==1]
    temp2 = ballData[ballData['match_id']==match][ballData['inning']==2]
    temp1['cumulative'] = temp1['total_runs'].cumsum()
    temp2['cumulative'] = temp2['total_runs'].cumsum()
    
    l.extend(list(temp1['cumulative']))
    l.extend(list(temp2['cumulative']))
    
ballData['cumulative']=l

In [ ]:
ballData['player_dismissed'] = ballData['player_dismissed'].fillna('0')
ballData.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,cumulative
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,0,NaN,NaN,4
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,4
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,0,NaN,NaN,6


In [ ]:
new = []
for row in range(178982) :
    if ballData.iloc[row]['player_dismissed']!='0' :
        new.append(1)
    else :
        new.append(0)
        
ballData['is_wicket'] = new

In [ ]:
l2 = []

for match in list(ballData['match_id'].unique()) :
    temp1 = ballData[ballData['match_id']==match][ballData['inning']==1]
    temp2 = ballData[ballData['match_id']==match][ballData['inning']==2]
    temp1['cumulative'] = temp1['is_wicket'].cumsum()
    temp2['cumulative'] = temp2['is_wicket'].cumsum()
    
    l2.extend(list(temp1['cumulative']))
    l2.extend(list(temp2['cumulative']))
ballData['cumulative_wickets']=l2

In [ ]:
h.head()

,id,season,city,date,team1,team2,toss_winner,toss_decision,result,dl_applied,winner,win_by_runs,win_by_wickets,player_of_match,venue,umpire1,umpire2,umpire3
0,1,2017,Hyderabad,2017-04-05,Sunrisers Hyderabad,Royal Challengers Bangalore,Royal Challengers Bangalore,field,normal,0,Sunrisers Hyderabad,35,0,Yuvraj Singh,"Rajiv Gandhi International Stadium, Uppal",AY Dandekar,NJ Llong,NaN
1,2,2017,Pune,2017-04-06,Mumbai Indians,Rising Pune Supergiant,Rising Pune Supergiant,field,normal,0,Rising Pune Supergiant,0,7,SPD Smith,Maharashtra Cricket Association Stadium,A Nand Kishore,S Ravi,NaN
2,3,2017,Rajkot,2017-04-07,Gujarat Lions,Kolkata Knight Riders,Kolkata Knight Riders,field,normal,0,Kolkata Knight Riders,0,10,CA Lynn,Saurashtra Cricket Association Stadium,Nitin Menon,CK Nandan,NaN
3,4,2017,Indore,2017-04-08,Rising Pune Supergiant,Kings XI Punjab,Kings XI Punjab,field,normal,0,Kings XI Punjab,0,6,GJ Maxwell,Holkar Cricket Stadium,AK Chaudhary,C Shamshuddin,NaN
4,5,2017,Bangalore,2017-04-08,Royal Challengers Bangalore,Delhi Daredevils,Royal Challengers Bangalore,bat,normal,0,Royal Challengers Bangalore,15,0,KM Jadhav,M Chinnaswamy Stadium,NaN,NaN,NaN


In [ ]:
ballData.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,cumulative,is_wicket,cumulative_wickets
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,0,NaN,NaN,4,0,0
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,4,0,0
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,0,NaN,NaN,6,0,0


In [ ]:
ballData['cumulative_wickets'].unique()

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [ ]:
h.rename(columns={'id':'match_id'}, inplace=True)

In [ ]:
ballData['over_ball'] = ballData['over'].astype(str) + '.' + ballData['ball'].astype(str)
ballData.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,cumulative,is_wicket,cumulative_wickets,over_ball
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,1.1
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,1.2
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,0,NaN,NaN,4,0,0,1.3
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,4,0,0,1.4
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,0,NaN,NaN,6,0,0,1.5


In [ ]:
l1 = list(ballData[ballData.inning==1]['match_id'].unique())
l2 = list(ballData[ballData.inning==2]['match_id'].unique())

removal = [x for x in l1 if x not in l2]  # Removing this data since match was abandoned

ballData.drop(ballData[ballData.match_id.isin(removal)].index, inplace=True)

ballData.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,cumulative,is_wicket,cumulative_wickets,over_ball
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,1.1
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,1.2
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,0,NaN,NaN,4,0,0,1.3
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,4,0,0,1.4
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,0,NaN,NaN,6,0,0,1.5


In [ ]:
addition = []
for i in list(ballData['match_id'].unique()) :
    i1 = ballData[ballData.match_id==i][ballData.inning==1]['cumulative'].iloc[-1]
    num1 = len(ballData[ballData.match_id==i][ballData.inning==1])
    i2 = ballData[ballData.match_id==i][ballData.inning==2]['cumulative'].iloc[-1]
    num2 = len(ballData[ballData.match_id==i][ballData.inning==2])
    addition.extend([i1]*num1)
    addition.extend([i2]*num2)

ballData['score'] = addition
ballData.head()

,match_id,inning,batting_team,bowling_team,over,ball,batsman,non_striker,bowler,is_super_over,wide_runs,bye_runs,legbye_runs,noball_runs,penalty_runs,batsman_runs,extra_runs,total_runs,player_dismissed,dismissal_kind,fielder,cumulative,is_wicket,cumulative_wickets,over_ball,score
0,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,1,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,1.1,207
1,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,2,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,0,0,0,1.2,207
2,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,3,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,4,0,4,0,NaN,NaN,4,0,0,1.3,207
3,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,4,DA Warner,S Dhawan,TS Mills,0,0,0,0,0,0,0,0,0,0,NaN,NaN,4,0,0,1.4,207
4,1,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1,5,DA Warner,S Dhawan,TS Mills,0,2,0,0,0,0,0,2,2,0,NaN,NaN,6,0,0,1.5,207


Merging the two relevant dataframes and picking out features to which the models will be applied

In [ ]:
data = pd.merge(h[['match_id','season','city','venue']], ballData, on='match_id')
data.columns

Index(['match_id', 'season', 'city', 'venue', 'inning', 'batting_team',
       'bowling_team', 'over', 'ball', 'batsman', 'non_striker', 'bowler',
       'is_super_over', 'wide_runs', 'bye_runs', 'legbye_runs', 'noball_runs',
       'penalty_runs', 'batsman_runs', 'extra_runs', 'total_runs',
       'player_dismissed', 'dismissal_kind', 'fielder', 'cumulative',
       'is_wicket', 'cumulative_wickets', 'over_ball', 'score'],
      dtype='object')

In [ ]:
dataset = data[['match_id','season','city','inning','batting_team','bowling_team','over_ball','cumulative','cumulative_wickets','score']]
dataset.head()

,match_id,season,city,inning,batting_team,bowling_team,over_ball,cumulative,cumulative_wickets,score
0,1,2017,Hyderabad,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.1,0,0,207
1,1,2017,Hyderabad,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.2,0,0,207
2,1,2017,Hyderabad,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.3,4,0,207
3,1,2017,Hyderabad,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.4,4,0,207
4,1,2017,Hyderabad,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.5,6,0,207


One-Hot encoding of the features containing categorical values

In [ ]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['city'], drop_first=True)], axis=1)
dataset = dataset.drop(['city'], axis=1)
dataset.head()

,match_id,season,inning,batting_team,bowling_team,over_ball,cumulative,cumulative_wickets,score,Ahmedabad,Bangalore,Bengaluru,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mohali,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam
0,1,2017,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.1,0,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2017,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.2,0,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1,2017,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.3,4,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1,2017,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.4,4,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1,2017,1,Sunrisers Hyderabad,Royal Challengers Bangalore,1.5,6,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['batting_team'], drop_first=True, prefix='bat')], axis=1)
dataset = dataset.drop(['batting_team'], axis=1)
dataset.head()

,match_id,season,inning,bowling_team,over_ball,cumulative,cumulative_wickets,score,Ahmedabad,Bangalore,Bengaluru,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mohali,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam,bat_Deccan Chargers,bat_Delhi Capitals,bat_Delhi Daredevils,bat_Gujarat Lions,bat_Kings XI Punjab,bat_Kochi Tuskers Kerala,bat_Kolkata Knight Riders,bat_Mumbai Indians,bat_Pune Warriors,bat_Rajasthan Royals,bat_Rising Pune Supergiant,bat_Rising Pune Supergiants,bat_Royal Challengers Bangalore,bat_Sunrisers Hyderabad
0,1,2017,1,Royal Challengers Bangalore,1.1,0,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,1,2017,1,Royal Challengers Bangalore,1.2,0,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,2017,1,Royal Challengers Bangalore,1.3,4,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,2017,1,Royal Challengers Bangalore,1.4,4,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
4,1,2017,1,Royal Challengers Bangalore,1.5,6,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
dataset = pd.concat([dataset, pd.get_dummies(dataset['bowling_team'], drop_first=True, prefix='bowl')], axis=1)
dataset = dataset.drop(['bowling_team'], axis=1)
dataset.head()

,match_id,season,inning,over_ball,cumulative,cumulative_wickets,score,Ahmedabad,Bangalore,Bengaluru,Bloemfontein,Cape Town,Centurion,Chandigarh,Chennai,Cuttack,Delhi,Dharamsala,Durban,East London,Hyderabad,Indore,Jaipur,Johannesburg,Kanpur,Kimberley,Kochi,Kolkata,Mohali,Mumbai,Nagpur,Port Elizabeth,Pune,Raipur,Rajkot,Ranchi,Sharjah,Visakhapatnam,bat_Deccan Chargers,bat_Delhi Capitals,bat_Delhi Daredevils,bat_Gujarat Lions,bat_Kings XI Punjab,bat_Kochi Tuskers Kerala,bat_Kolkata Knight Riders,bat_Mumbai Indians,bat_Pune Warriors,bat_Rajasthan Royals,bat_Rising Pune Supergiant,bat_Rising Pune Supergiants,bat_Royal Challengers Bangalore,bat_Sunrisers Hyderabad,bowl_Deccan Chargers,bowl_Delhi Capitals,bowl_Delhi Daredevils,bowl_Gujarat Lions,bowl_Kings XI Punjab,bowl_Kochi Tuskers Kerala,bowl_Kolkata Knight Riders,bowl_Mumbai Indians,bowl_Pune Warriors,bowl_Rajasthan Royals,bowl_Rising Pune Supergiant,bowl_Rising Pune Supergiants,bowl_Royal Challengers Bangalore,bowl_Sunrisers Hyderabad
0,1,2017,1,1.1,0,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
1,1,2017,1,1.2,0,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,1,2017,1,1.3,4,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,1,2017,1,1.4,4,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,1,2017,1,1.5,6,0,207,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
X = pd.concat([dataset.iloc[:, :6], dataset.iloc[:,7:]], axis=1)
y = dataset.iloc[:,6]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [ ]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

**Linear Regression to predict final Score**

In [ ]:
lr = LinearRegression()

model = lr.fit(X_train, y_train)

In [ ]:
model.score(X_train, y_train)

0.46263032605633836

In [ ]:
model.score(X_test, y_test)

0.46918632842297325

In [ ]:
prediction = model.predict(X_test)

d = [[a,b] for a,b in zip(y_test, prediction)]
op = pd.DataFrame(d, columns=['Y_Test','Prediction'])
op

,Y_Test,Prediction
0,140,156.226139
1,181,148.142167
2,142,126.037925
3,153,146.429820
4,158,156.776198
...,...,...
35754,105,138.567622
35755,81,133.628743
35756,171,178.288099
35757,131,126.166316


In [ ]:
rmse = 0
for y,pred in zip(list(op['Y_Test']), list(op['Prediction'])) :
    rmse += (y-pred)**2
remse = rmse**0.5
print('Root Mean Square Error :', rmse)

Root Mean Square Error : 17255787.13534094


**Gradient Boosting Reegressor to predict Final Score**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid_rf = {
              #"n_estimators": [200,300],
              "learning_rate": [0.25,0.5,0.75],
              "max_depth": [10,14,18],           
             }         
gr_rf = GridSearchCV(GradientBoostingRegressor(),param_grid_rf,cv=5,verbose=2)
gr_rf.fit(X_train, y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END ...................learning_rate=0.25, max_depth=10; total time= 2.4min
[CV] END ...................learning_rate=0.25, max_depth=10; total time= 1.0min
[CV] END ...................learning_rate=0.25, max_depth=10; total time= 1.0min
[CV] END ...................learning_rate=0.25, max_depth=10; total time= 1.1min
[CV] END ...................learning_rate=0.25, max_depth=10; total time= 1.1min
[CV] END ...................learning_rate=0.25, max_depth=14; total time= 1.4min
[CV] END ...................learning_rate=0.25, max_depth=14; total time= 1.4min
[CV] END ...................learning_rate=0.25, max_depth=14; total time= 1.5min
[CV] END ...................learning_rate=0.25, max_depth=14; total time= 1.5min
[CV] END ...................learning_rate=0.25, max_depth=14; total time= 1.5min
[CV] END ...................learning_rate=0.25, max_depth=18; total time= 1.7min
[CV] END ...................learning_rate=0.25, m

GridSearchCV(cv=5, estimator=GradientBoostingRegressor(),
             param_grid={'learning_rate': [0.25, 0.5, 0.75],
                         'max_depth': [10, 14, 18]},
             verbose=2)

In [ ]:
gr_rf.best_params_

{'learning_rate': 0.25, 'max_depth': 14}

In [ ]:
rf_best = GradientBoostingRegressor(random_state=300,
                                    n_estimators =5000,
                                    learning_rate=gr_rf.best_params_['learning_rate'],
                                    max_depth = gr_rf.best_params_['max_depth'],
                                    )
rf_best.fit(X_train,y_train)
y_pred = rf_best.predict(X_test)
score = rf_best.score(X_test,y_test)

rmse = 0
for y,pred in zip(y_test, y_pred) :
    rmse += (y-pred)**2
remse = rmse**0.5

print("Score :", score)
print('Root Mean Square Error :', rmse)

Score : 0.9932668396728275
Root Mean Square Error : 218882.79743930994


In [ ]:
pd.DataFrame([[a,b] for a,b in zip(y_test, y_pred)], columns=['Y_Test','Prediction'])

,Y_Test,Prediction
0,140,140.035340
1,181,177.979856
2,142,141.895009
3,153,152.724744
4,158,156.555158
...,...,...
35754,105,105.536838
35755,81,83.475031
35756,171,171.005823
35757,131,131.136489


**Random Forest Regressor to predict Final Score**

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid_rf2 = {
              "n_estimators": [100,300,500],
              "max_depth": [10,50,100],           
             }         
gr_rf2 = GridSearchCV(RandomForestRegressor(),param_grid_rf2,cv=5,verbose=2)
gr_rf2.fit(X_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] END .....................max_depth=10, n_estimators=100; total time=  39.0s
[CV] END .....................max_depth=10, n_estimators=100; total time=  40.3s
[CV] END .....................max_depth=10, n_estimators=100; total time=  39.6s
[CV] END .....................max_depth=10, n_estimators=100; total time=  38.6s
[CV] END .....................max_depth=10, n_estimators=100; total time=  38.5s
[CV] END .....................max_depth=10, n_estimators=300; total time= 1.9min
[CV] END .....................max_depth=10, n_estimators=300; total time= 1.9min
[CV] END .....................max_depth=10, n_estimators=300; total time= 1.9min
[CV] END .....................max_depth=10, n_estimators=300; total time= 1.9min
[CV] END .....................max_depth=10, n_estimators=300; total time= 1.9min
[CV] END .....................max_depth=10, n_estimators=500; total time= 3.2min
[CV] END .....................max_depth=10, n_est

GridSearchCV(cv=5, estimator=RandomForestRegressor(),
             param_grid={'max_depth': [10, 50, 100],
                         'n_estimators': [100, 300, 500]},
             verbose=2)

In [ ]:
gr_rf2.best_params_

{'max_depth': 100, 'n_estimators': 500}

In [ ]:
rf_best2 = RandomForestRegressor(random_state=300,max_features=None,
                                    n_estimators=gr_rf2.best_params_['n_estimators'],
                                    max_depth = gr_rf2.best_params_['max_depth'], verbose=2
                                    )
rf_best2.fit(X_train,y_train)
y_pred = rf_best2.predict(X_test)
score = rf_best2.score(X_test,y_test)

rmse = 0
for y,pred in zip(y_test, y_pred) :
    rmse += (y-pred)**2
remse = rmse**0.5

print("Score :", score)
print('Root Mean Square Error :', rmse)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


building tree 1 of 500


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.6s remaining:    0.0s


building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
building tree 43 of 500


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  7.9min finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    5.8s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


Score : 0.9882340499270817
Root Mean Square Error : 380756.4830640001


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    5.5s finished


In [ ]:
pd.DataFrame([[a,b] for a,b in zip(y_test, y_pred)], columns=['Y_Test','Prediction'])

,Y_Test,Prediction
0,177,179.376
1,178,177.304
2,146,146.000
3,189,191.386
4,195,194.774
...,...,...
35754,172,173.246
35755,221,221.000
35756,165,164.744
35757,102,102.664
